<a href="https://colab.research.google.com/github/nyp-sit/iti107/blob/main/session-2/xception_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Augmentation

Welcome to this week's programming exercise. In this week's exercises, we will learn how to improve our model performance using common tecniques such as data augmentation and transfer learning. 

Data augmentation is typically used when you have a small set of training samples. It allows you to increase your number of samples by generating artificial samples, either based on some random transformation of your existing samples, or by some statistical means. The larger training samples can help the model to generalize better. 


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt

### Create train and validation dataset

We will go ahead and prepare our train and validation dataset (the cats vs dogs dataset) as before. 

In [ ]:
import os 

dataset_URL = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/cats_and_dogs_subset.tar.gz'
tf.keras.utils.get_file(origin=dataset_URL, extract=True, cache_dir='.')
dataset_folder = os.path.join('datasets', 'cats_and_dogs_subset')

In [ ]:
batch_size = 16
image_size = (128,128)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_folder,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_folder,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)

## Data Augmentation 

Since tensorflow 2.2, Keras introduces new types of layers for doing image data augmentation, such as Random Cropping, Random Flipping, etc. Previously, we have to depend on ImageDataGenerator() (which is a lot slower) to do so. 

In the code below, we create a Sequential model to add the image augmentation layer: `RandomRotation()`. The value `0.1' refers to the maximum rotation angle in both clock-wise and anti-clockwise direction. You can find out more info from the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomRotation)

In [ ]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomRotation(0.1),
    ]
)

To see the effects of data augmentation, let us apply our data_augmentation layer to a sample image.

In [ ]:
images, _ = next(train_ds.take(1).as_numpy_iterator())
sample_image = images[0]/255.
plt.imshow(sample_image)
sample_image = tf.expand_dims(sample_image, 0)
print(sample_image.shape)

In [ ]:
plt.figure(figsize=(8, 4))
for i in range(8):
    augmented_image = data_augmentation(sample_image)
    ax = plt.subplot(2, 4, i + 1)
    plt.imshow(augmented_image[0])
    plt.axis("off")

**Exercise 1:**

Modify the code above to add in random Horizontal flip Choose the appropriate values for the contrast and cropping factor.

<details><summary>Click here for answer</summary>

```python
    
data_augmentation = keras.Sequential(
    [
        layers.RandomRotation(0.1),
        layers.RandomFlip("horizontal")
    ]
)    
```
    
</details>

In [ ]:
## TODO: Modify the code to add data augmentation
data_augmentation = keras.Sequential(
        [
            tf.keras.layers.RandomRotation(0.1),
            tf.keras.layers.RandomFlip("horizontal")
        ]
    )

## Build the model

Previously we have built the mini-Xception network and use it on our small cats and dogs dataset. It performs slightly better than the first simple model we built, but not much of an improvement. Here we will use the same network but adds in the data augmnetation to see if our model can be improved further. 

The following codes are same as previous xception network that you have coded. 


In [ ]:
def xception_block(x, depth): 
    # save input to be used as skip connection
    residual = x
    
    # add the first separable convolutional 2D layer (as well as the batch normalization and activation layer)
    x = keras.layers.SeparableConv2D(depth, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    
    # add the second separable convolutional 2D layer (as well as the batch normalization BUT without activation layer)
    x = keras.layers.SeparableConv2D(depth, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    
    # add the maxpooling 2d layer, with stride of 2
    x = keras.layers.MaxPool2D(3, strides=2, padding='same')(x)
    
    # adjust the size and depth using 1x1 convolution to match the output from last maxpooling layer. Use strides=2 to match the output from maxpooling
    residual = keras.layers.Conv2D(depth, 1, strides=2, padding='same')(residual)
    
    # add back the residual to the output from maxpooling layer
    x = keras.layers.add( [x, residual] )
    
    # add the activation layer
    x = keras.layers.Activation('relu')(x)
    
    return x # Set aside next residual


**Exercise 2:**

Modify the code in `make_model()` to apply data augmention layers you have created earlier. Where should you place your augmentation layer?  

<details><summary>Click here for answer</summary>

```python
def make_model(input_shape, num_classes): 
    inputs = keras.Input(shape=input_shape)    
    
    ## Add your augmentation layers here !! 
    x = data_augmentation(inputs) 

    x = layers.Rescaling(1.0 / 255)(inputs)

    ## the rest of the codes.... 
    
    return keras.Model(inputs, outputs)    
```
    
</details>

In [ ]:
def make_model(input_shape, num_classes): 

    inputs = keras.Input(shape=input_shape)

    ## Add your augmentation layers here !! 
    x = data_augmentation(inputs) 

    # add resclaing 
    x = keras.layers.Rescaling(1./255)(x)
    
    # Entry blocks

    # 1st conv2d with strides = 2
    x = keras.layers.Conv2D(32, 3, strides=2, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)

    # 2nd conv2d with strides = 1
    x = keras.layers.Conv2D(64, 3, strides=1, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    
    # build a series of xception blocks with different depth
    for depth in [128, 256, 512, 728]:
        x = xception_block(x, depth)
    
    # add SeparableConv2D
    x = keras.layers.SeparableConv2D(1024, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    
    # add Global Average Pooling layer before connecting to Dense layer 
    x = keras.layers.GlobalAveragePooling2D()(x)
    
    x = keras.layers.Dropout(0.5)(x)
    
    outputs = keras.layers.Dense(1, activation='sigmoid')(x)
    
    return keras.Model(inputs, outputs)

In [ ]:
model = make_model(input_shape= image_size + (3,), num_classes=2)

## Train the model

Let's train our new model with the data augmentation layer.  We increase our training epochs to 50 to give our model more chances to see the augmented images.

In [ ]:
def create_tb_callback(): 

    import os
    
    root_logdir = os.path.join(os.curdir, "tb_logs")

    def get_run_logdir():    # use a new directory for each run
        
        import time
        
        run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
        return os.path.join(root_logdir, run_id)

    run_logdir = get_run_logdir()

    tb_callback = tf.keras.callbacks.TensorBoard(run_logdir)

    return tb_callback

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="best_checkpoint",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


# compile our model with loss and optimizer 
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)


model.fit(
    train_ds, epochs=50, 
    validation_data=val_ds,
    callbacks=[model_checkpoint_callback, create_tb_callback()]
)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs

In [ ]:
best_checkpoint = 'best_checkpoint'

model.load_weights(best_checkpoint)
model.evaluate(val_ds)